# DS Automation Assignment

Using our prepared churn data from week 2:
- use pycaret to find an ML algorithm that performs best on the data
    - Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.
- save the model to disk
- create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
    - your Python file/function should print out the predictions for new data (new_churn_data.csv)
    - the true values for the new data are [1, 0, 0, 1, 0] if you're interested
- test your Python module and function with the new data, new_churn_data.csv
- write a short summary of the process and results at the end of this notebook
- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox

*Optional* challenges:
- return the probability of churn for each new prediction, and the percentile where that prediction is in the distribution of probability predictions from the training dataset (e.g. a high probability of churn like 0.78 might be at the 90th percentile)
- use other autoML packages, such as TPOT, H2O, MLBox, etc, and compare performance and features with pycaret
- create a class in your Python module to hold the functions that you created
- accept user input to specify a file using a tool such as Python's `input()` function, the `click` package for command-line arguments, or a GUI
- Use the unmodified churn data (new_unmodified_churn_data.csv) in your Python script. This will require adding the same preprocessing steps from week 2 since this data is like the original unmodified dataset from week 1.

In [1]:
import pandas as pd

df = pd.read_csv('data/prepared_churn_data.csv', index_col='customerID')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,NoContract,ElectronicCheck,Difference,MonthlyRatio
customerID,,,,,,,,,,,
7590-VHVEG,1.0,0,0,0,29.85,29.85,0,1,1,0.00,29.850000
5575-GNVDE,34.0,1,1,1,56.95,1889.50,0,0,0,-46.80,55.573529
3668-QPYBK,2.0,1,0,1,53.85,108.15,1,1,0,0.45,54.075000
7795-CFOCW,45.0,0,1,2,42.30,1840.75,0,0,0,-62.75,40.905556
9237-HQITU,2.0,1,0,0,70.70,151.65,1,1,1,10.25,75.825000
...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,24.0,1,1,1,84.80,1990.50,0,0,0,-44.70,82.937500
2234-XADUH,72.0,1,1,3,103.20,7362.90,0,0,0,-67.50,102.262500
4801-JZAZL,11.0,0,0,0,29.60,346.45,0,1,1,20.85,31.495455


In [2]:
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

In [11]:
automl = setup(df, target='Churn')

,Description,Value
0,session_id,7633
1,Target,Churn
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(7043, 11)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


In [12]:
automl[6]

,tenure,MonthlyCharges,TotalCharges,Difference,PhoneService_1,Contract_0,Contract_1,Contract_2,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3,ElectronicCheck_1
customerID,,,,,,,,,,,,
7590-VHVEG,1.0,29.850000,29.850000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5575-GNVDE,34.0,56.950001,1889.500000,-46.799999,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3668-QPYBK,2.0,53.849998,108.150002,0.450000,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7795-CFOCW,45.0,42.299999,1840.750000,-62.750000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
9237-HQITU,2.0,70.699997,151.649994,10.250000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,24.0,84.800003,1990.500000,-44.700001,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2234-XADUH,72.0,103.199997,7362.899902,-67.500000,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4801-JZAZL,11.0,29.600000,346.450012,20.850000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7901,0.8289,0.5084,0.6314,0.5624,0.4267,0.4315,0.0090
gbc,Gradient Boosting Classifier,0.7892,0.8315,0.4939,0.6356,0.5541,0.4194,0.4261,0.0530
catboost,CatBoost Classifier,0.7874,0.8269,0.4894,0.6326,0.5505,0.4145,0.4211,0.4390
ridge,Ridge Classifier,0.7862,0.0000,0.4445,0.6423,0.5245,0.3929,0.4043,0.0040
lda,Linear Discriminant Analysis,0.7848,0.8194,0.5084,0.6166,0.5565,0.4164,0.4202,0.0050
ada,Ada Boost Classifier,0.7842,0.8264,0.4734,0.6248,0.5374,0.4006,0.4077,0.0240
lightgbm,Light Gradient Boosting Machine,0.7789,0.8214,0.4901,0.6070,0.5415,0.3982,0.4026,0.0460
xgboost,Extreme Gradient Boosting,0.7755,0.8095,0.4916,0.5961,0.5380,0.3917,0.3953,0.1640
rf,Random Forest Classifier,0.7673,0.7995,0.4574,0.5844,0.5117,0.3622,0.3677,0.0550
et,Extra Trees Classifier,0.7611,0.7814,0.4704,0.5655,0.5120,0.3559,0.3594,0.0490


In [14]:
best_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=7633, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
df.iloc[-2:-1]

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,NoContract,ElectronicCheck,Difference,MonthlyRatio
customerID,,,,,,,,,,,
8361-LTMKD,4.0,1,0,1,74.4,306.6,1,1,0,9.0,76.65


In [16]:
df.iloc[-1]

tenure               66.000000
PhoneService          1.000000
Contract              2.000000
PaymentMethod         2.000000
MonthlyCharges      105.650000
TotalCharges       6844.500000
Churn                 0.000000
NoContract            0.000000
ElectronicCheck       0.000000
Difference         -128.400000
MonthlyRatio        103.704545
Name: 3186-AJIEK, dtype: float64

In [17]:
predict_model(best_model, df.iloc[-2:-1])

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,NoContract,ElectronicCheck,Difference,MonthlyRatio,Label,Score
customerID,,,,,,,,,,,,,
8361-LTMKD,4.0,1,0,1,74.4,306.6,1,1,0,9.0,76.65,0,0.5174


In [18]:
save_model(best_model, 'LR')

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='Churn',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strate...
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  LogisticRegression(C=1.0, class_weight=None, dual=False,
                 

In [20]:
import pickle

with open('LR_model.pk', 'wb') as f:
    pickle.dump(best_model, f)

In [21]:
with open('LR_model.pk', 'rb') as f:
    loaded_model = pickle.load(f)

In [22]:
new_data = df.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)

In [ ]:
new_data = df.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)
loaded_model.predict(new_data)

In [23]:
loaded_lr = load_model('LR')

Transformation Pipeline and Model Successfully Loaded


In [24]:
predict_model(loaded_lr, new_data)

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,NoContract,ElectronicCheck,Difference,MonthlyRatio,Label,Score
customerID,,,,,,,,,,,,
8361-LTMKD,4.0,1,0,1,74.4,306.6,1,0,9.0,76.65,0,0.5174


In [31]:
from IPython.display import Code

Code('predict_churn.py')

import pandas as pd
from pycaret.classification import predict_model, load_model

def load_data(filepath):
    """
    Loads diabetes data into a DataFrame from a string filepath.
    """
    df = pd.read_csv(filepath, index_col='customerID')
    return df


def make_predictions(df):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    """
    model = load_model('LR')
    predictions = predict_model(model, data=df)
    predictions.rename({'Label': 'Churn_prediction'}, axis=1, inplace=True)
    predictions['Churn_prediction'].replace({1: 'Churn', 0: 'No Churn'},
                                            inplace=True)
    return predictions['Churn_prediction']


if __name__ == "__main__":
    df = load_data('data/new_churn_data.csv')
    
    df['NoContract'] = df['Contract']   # create a new column based on contract
    df['NoContract'] = df['NoContract'].replace({2 : 1})
    df['NoContract'] = 1 - df['NoContract']   # kind of wacky but it swaps
    
    df['ElectronicCheck'] = df['PaymentMethod']   # create a new column based on payment
    df['ElectronicCheck'] = df['ElectronicCheck'].replace({2 : 1, 3 : 1})
    df['ElectronicCheck'] = 1 - df['ElectronicCheck']   # kind of wacky but it swaps
    
    df['Difference'] = df['TotalCharges'] - (df['MonthlyCharges'] * df['tenure'])
    
    df['MonthlyRatio'] = df['TotalCharges'] / df['tenure']

    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

In [32]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded
predictions:
customerID
9305-CKSKC    No Churn
1452-KNGVK    No Churn
6723-OKKJM    No Churn
7832-POPKP    No Churn
6348-TACGU       Churn
Name: Churn_prediction, dtype: object


# Summary

Write a short summary of the process and results here.